# Music
<img src="Music/Y4_Large_reduc.jpg" align="right" width="64" height="64" />

Zen's Music Playround

In [40]:
library = "/home/fruitbat/Zen/Music/"

lib = MusicLib(library)
# print("Artists = {0}".format(lib.artists()))
artists = lib.get_random_artists(5)
# print("Random Artists = {0}".format(artists))
for artist in artists:
    albums = lib.albums(artist)
    print("{0} has {1} album".format(artist, len(albums)))
    [print("    {0}".format(album)) for album in albums]

Tom Waits has 1 album
    Bone Machine
Roots Manuva has 1 album
    Run Come Save Me
Malmaison has 1 album
    Ecoutez Et Repetez Deux 2
Sibot has 1 album
    The Real Estate Agents
Kings X has 1 album
    The Best Of Kings X


In [41]:
import ipywidgets as widgets

def on_click(_button):
    print("button click")

button = widgets.Button(
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
button.on_click(on_click)
button


In [15]:
from os import listdir
from os.path import join, isdir
from random import sample

class MusicLib(object):
    """
    Class for fetching information about our music library, as an experiment
    in jupyter
    """
   
    def __init__(self, path):
       self.path = path 
    
    def artists(self):
        """ Return a list of lartists. """
        return [name for name in listdir(self.path) if isdir(join(self.path, name))]

    def get_random_artists(self, number):
        """ Return a list of *number* random artists. """
        artists = self.artists()
        return sample(artists, number)
    
    def albums(self, artist):
        """ Return a list of albums for the *artist*. """
        path = join(self.path, artist)
        return [name for name in listdir(path) if isdir(join(path, name))]
    